# sm'

In [ ]:
// // test

open testing

()



In [ ]:
open rust_operators

()



## types

In [ ]:
inl types () =
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"str\")>]\n#endif\ntype Str = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"base64::DecodeError\")>]\n#endif\ntype base64_DecodeError = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"borsh::io::Error\")>]\n#endif\ntype borsh_io_Error = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"js_sys::JsString\")>]\n#endif\ntype js_sys_JsString = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"serde_json::Error\")>]\n#endif\ntype serde_json_Error = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"serde_json::Value\")>]\n#endif\ntype serde_json_Value = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"serde_wasm_bindgen::Error\")>]\n#endif\ntype serde_wasm_bindgen_Error = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::ffi::OsStr\")>]\n#endif\ntype std_ffi_OsStr = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::ffi::OsString\")>]\n#endif\ntype std_ffi_OsString = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::fmt::Display<$0>\")>]\n#endif\ntype std_fmt_Display<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::str::Utf8Error\")>]\n#endif\ntype std_str_Utf8Error = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::string::FromUtf8Error\")>]\n#endif\ntype std_string_FromUtf8Error = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::string::String\")>]\n#endif\ntype std_string_String = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::slice::Windows<$0>\")>]\n#endif\ntype std_slice_Windows<'T> = class end"

()



In [ ]:
inl types () =
    rust.types ()
    types ()

()



In [ ]:
nominal display t = $"std_fmt_Display<`t>"
nominal str = $"Str"
nominal base64_decode_error = $"base64_DecodeError"
nominal borsh_io_error = $"borsh_io_Error"
nominal utf8_error = $"std_str_Utf8Error"
nominal from_utf8_error = $"std_string_FromUtf8Error"
nominal json_value = $"serde_json_Value"
nominal json_error = $"serde_json_Error"
nominal serde_wasm_bindgen_error = $"serde_wasm_bindgen_Error"
nominal js_string = $"js_sys_JsString"
nominal os_str = $"std_ffi_OsStr"
nominal os_string = $"std_ffi_OsString"
nominal std_string = $"std_string_String"

()



## contains

In [ ]:
inl contains (value : string) (s : string) : bool =
    $"!s.Contains !value"

()



## ends_with

In [ ]:
inl ends_with (value : string) (s : string) : bool =
    $"!s.EndsWith !value"

()



## last_index_of

In [ ]:
inl last_index_of (search : string) (s : string) : i32 =
    $"!s.LastIndexOf !search"

()



## pad_left

In [ ]:
inl pad_left (total_width : i32) (padding_char : char) (s : string) : string =
    $"!s.PadLeft (!total_width, !padding_char)"

()



## pad_right

In [ ]:
inl pad_right (total_width : i32) (padding_char : char) (s : string) : string =
    $"!s.PadRight (!total_width, !padding_char)"

()



In [ ]:
// // test

"123"
|> pad_right 6 ' '
|> _assert_eq "123   "

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "123"
    let v1 : string = v0.PadRight (6, ' ')
    let v3 : bool = v1 = "123   "
    let v5 : bool =
        if v3 then
            true
        else
            method1(v3)
    let v6 : string = "assert_eq"
    let v7 : string = "123   "
    let v8 : string = $"{v6} / actual: %A{v1} / expected: %A{v7}"
    System.Console.WriteLine v8
    let v9 : bool = v5 = false
    if v9 then
        failwith<unit> v8
method0()

assert_eq / actual: "123   " / expected: "123   "


## starts_with

In [ ]:
inl starts_with (value : string) (s : string) : bool =
    $"!s.StartsWith !value"

()



## substring

In [ ]:
inl substring (start : i32) (len : i32) (str : string) : string =
    $"!str.Substring (!start, !len)"

()



## to_lower

In [ ]:
inl to_lower (input : string) : string =
    $"!input.ToLower ()"

()



## to_upper

In [ ]:
inl to_upper (input : string) : string =
    $"!input.ToUpper ()"

()



## trim

In [ ]:
inl trim (input : string) : string =
    $"!input.Trim ()"

()



## trim_end

In [ ]:
inl trim_end (trim_chars : array_base char) (input : string) : string =
    $"!input.TrimEnd !trim_chars"

()



## trim_start

In [ ]:
inl trim_start (trim_chars : array_base char) (input : string) : string =
    $"!input.TrimStart !trim_chars"

()



## length

In [ ]:
inl length forall dim. (input : string) : dim =
    $"!input |> String.length"

()



## slice

In [ ]:
inl slice from to s : string =
    sm.slice s { from to }

()



## range

In [ ]:
union range dim =
    | Start : dim
    | End : dim -> dim

inl range start end s =
    inl start, end =
        match start, end with
        | Start start, End fn =>
            start, s |> length |> fn
        | End start_fn, End end_fn =>
            inl len = s |> length
            start_fn len, end_fn len
    s |> slice start end

()



## raw_string_literal

In [ ]:
inl raw_string_literal (s : string) : rust.ref' str =
    !\($"\"r#\\\"\" + !s + \"\\\"#\"")

()



## raw_string_literal_static

In [ ]:
inl raw_string_literal_static (s : string) : rust.static_ref' str =
    !\($"\"r#\\\"\" + !s + \"\\\"#\"")

()



## (~#)

In [ ]:
inl (~#) s =
    s |> raw_string_literal

()



## (~##)

In [ ]:
inl (~##) s =
    s |> raw_string_literal_static

()



## include_str

In [ ]:
inl include_str (path : string) : rust.ref' str =
    !\($'"include_str\!(\\\"" + !path + "\\\")"')

()



## as_str

In [ ]:
inl as_str (s : string) : rust.ref' str =
    !\\(s, $'"fable_library_rust::String_::LrcStr::as_str(&$0)"')

()



## from_std_string

In [ ]:
inl from_std_string (str : std_string) : string =
    !\\(str, $'"fable_library_rust::String_::fromString($0)"')

()



## ref_to_std_string

In [ ]:
inl ref_to_std_string (str : rust.ref' str) : std_string =
    !\\(str, $'"String::from($0)"')

()



## to_std_string

In [ ]:
inl to_std_string (s : string) : std_string =
    inl s = join s
    s |> as_str |> ref_to_std_string

()



## as_str_std

In [ ]:
inl as_str_std (s : std_string) : rust.ref' str =
    inl s = join s
    !\($'"!s.as_str()"')

()



## into_boxed_str

In [ ]:
inl into_boxed_str (s : std_string) : rust.box str =
    !\($'"!s.into_boxed_str()"')

()



## as_os_ref

In [ ]:
inl as_os_ref (s : os_string) : rust.ref' os_str =
    !\\(s, $'"$0.as_ref()"')

()



## to_os_string

In [ ]:
inl to_os_string (s : rust.ref' os_str) : os_string =
    !\\(s, $'"$0.to_os_string()"')

()



## os_to_str

In [ ]:
inl os_to_str (s : os_string) : optionm'.option' (rust.ref' str) =
    !\\(s, $'"$0.to_str()"')

()



## from_os_str_ref

In [ ]:
inl from_os_str_ref s =
    s
    |> to_os_string
    |> os_to_str
    |> optionm'.unwrap
    |> ref_to_std_string
    |> from_std_string

()



## obj_to_string

In [ ]:
inl obj_to_string x : string =
    x |> $'_.ToString()'

()



## to_string any

In [ ]:
instance to_string any =
    obj_to_string

()



## to_string result t u

In [ ]:
instance to_string result t u = fun x =>
    real
        open rust
        typecase (t * u) with
        | string * string =>
            match x with
            | Ok x => x
            | Error x => $'"sm\'.to_string result / Error: " + !x + ""' : string
        | std_string * std_string =>
            match x with
            | Ok x => from_std_string x
            | Error x => $'"sm\'.to_string result / Error: " + string !x + ""' : string
        | _ => obj_to_string `u x

()



## format''

In [ ]:
inl format'' (format : string) : std_string =
    !\($'@@$"format\!(" + !format + ")"')

()



## format'

In [ ]:
inl format' x : std_string =
    run_target function
        | Rust _ => fun () =>
            !\\(x, $'@@$"format\!(""{{}}"", $0)"')
        | _ => fun () => null ()

()



## format_debug

In [ ]:
inl format_debug x : string =
    $'$"%A{!x}"'

()



## format_debug'

In [ ]:
inl format_debug' x : std_string =
    !\\(x, $'@@$"format\!(""{{:?}}"", $0)"')

()



## format_pretty'

In [ ]:
inl format_pretty' x : std_string =
    !\\(x, $'@@$"format\!(""{{:#?}}"", $0)"')

()



## format_exception

In [ ]:
inl format_exception (ex : exn) : string =
    run_target function
        | Fsharp (Native) => fun () => $'$"{!ex.GetType ()}: {!ex.Message}"'
        | _ => fun () => ex |> format_debug

()



In [ ]:
// // test

fun () => failwith "test"
|> _throws
|> optionm.map sm'.format_exception
|> _assert_eq (Some "System.Exception: test")

type [<Struct>] US0 =
    | US0_0 of f0_0 : exn
    | US0_1
and [<Struct>] US1 =
    | US1_0 of f0_0 : string
    | US1_1
let rec closure0 () () : unit =
    failwith<unit> "test"
and closure1 () (v0 : exn) : US0 =
    US0_0(v0)
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (unit -> unit) = closure0()
    let v1 : US0 = US0_1
    let v2 : (exn -> US0) = closure1()
    let v3 : US0 = try v0 (); v1 with ex -> ex |> v2
    let v17 : US1 =
        match v3 with
        | US0_1 -> (* None *)
            US1_1
        | US0_0(v4) -> (* Some *)
            let v5 : string option = None
            let v6 : bool = true in let mutable _v5 = v5
            
#if FABLE_COMPILER || WASM || CONTRACT
            
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
            let v7 : string = $"%A{v4}"
            v7
#endif
            
#if FABLE_COMPILER_RUST && WASM
            let v8 : string = $"%A{v4}"
            v8
#endif
            
#if FABLE_COMPILER_RUST && CONTRA

## replace

In [ ]:
inl replace (old_value : string) (new_value : string) (s : string) : string =
    $"!s.Replace (!old_value, !new_value)"

()



## replace_regex

In [ ]:
inl replace_regex (pattern : string) (replacement : string) (s : string) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $"System.Text.RegularExpressions.Regex.Replace (!s, !pattern, !replacement)"
        | Rust (Native) => fun () =>
            open rust_operators
            inl pattern = join pattern
            inl s = join s
            inl replacement = join replacement
            !\\((pattern, s, replacement), $'$"&regex::Regex::new(&$0).unwrap().replace_all(&$1, &*$2)"')
            |> ref_to_std_string
            |> from_std_string
        | _ => fun () => null ()

()



In [ ]:
// // test

" 123"
|> replace_regex "\\s\\w2" ""
|> _assert_eq "3"

let rec method1 () : string =
    let v0 : string = "\\s\\w2"
    v0
and method2 () : string =
    let v0 : string = " 123"
    v0
and method3 () : string =
    let v0 : string = ""
    v0
and method4 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string option = None
    let v1 : bool = true in let mutable _v0 = v0
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : string = method1()
    let v3 : string = method2()
    let v4 : string = method3()
    let v5 : string = $"&regex::Regex::new(&$0).unwrap().replace_all(&$1, &*$2)"
    let v6 : Ref<Str> = Fable.Core.RustInterop.emitRustExpr struct (v2, v3, v4) v5
    let v7 : string = "String::from($0)"
    let v8 : std_string_String = Fable.Core.RustInterop.emitRustExpr v6 v7
    let v9 : string = "fable_library_rust::String_::fromString($0)"
    let v10 : string = Fable.Core.RustInterop.emitRustExpr v8 v9
    v10
#endif
    
#if FABLE_COMPILER_RUST && WASM
    let 

In [ ]:
// // test
// // rust=

types ()
" 123"
|> replace_regex "\\s\\w2" ""
|> _assert_eq "3"

.rs output:
assert_eq / actual: "3" / expected: "3"




.fsx:
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type ActionFn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $

## serialize

In [ ]:
inl serialize forall t. (x : t) : resultm.result' std_string json_error =
    !\($'"serde_json::to_string(&!x)"')

()



## deserialize

In [ ]:
inl deserialize forall t. (json : string) : resultm.result' t std_string =
    inl json = join json
    inl json = json |> as_str
    !\($'"serde_json::from_str(&!json)"')
    |> resultm.map_error' fun (x : json_error) => x |> format'

()



## borsh_deserialize

In [ ]:
inl borsh_deserialize forall t. (data : array_base u8) : resultm.result' t std_string =
    inl data = data |> am'.as_slice
    !\($'"let mut !data = !data"')
    inl result = !\($'"borsh::BorshDeserialize::deserialize(&mut !data)"')
    result
    |> resultm.map_error' fun (x : borsh_io_error) => x |> format'

()



## deserialize_vec

In [ ]:
inl deserialize_vec (value : json_value) : resultm.result' (am'.vec u8) std_string =
    inl value = join value
    !\($'"serde_json::from_value(!value)"')
    |> resultm.map_error' fun (x : json_error) => x |> format'

()



## encode_uri_component

In [ ]:
inl encode_uri_component (s : std_string) : js_string =
    !\($'"js_sys::encode_uri_component(&!s)"')

()



## strip_prefix

In [ ]:
inl strip_prefix (prefix : char) (s : std_string) : optionm'.option' (rust.ref' str) =
    inl s = join s
    !\($'"!s.strip_prefix(!prefix)"')

()



## str_from_utf8

In [ ]:
inl str_from_utf8 (bytes : rust.ref' (am'.slice u8)) : resultm.result' (rust.ref' str) utf8_error =
    !\($'"std::str::from_utf8(!bytes)"')

()



## string_from_utf8

In [ ]:
inl string_from_utf8 (bytes : am'.vec u8) : resultm.result' std_string from_utf8_error =
    !\($'"std::string::String::from_utf8(!bytes)"')

()



## base64_decode

In [ ]:
inl base64_decode (s : std_string) : result std_string std_string =
    fun () =>
        inl s = join s
        inl bytes : resultm.result' (am'.vec u8) base64_decode_error =
            !\($'"base64::Engine::decode(&base64::engine::general_purpose::STANDARD, !s)"')
        bytes
        |> resultm.map_error' format'
        |> resultm.try'
        |> string_from_utf8
        |> resultm.map_error' format'
    |> fun x =>
        join x ()
        |> resultm.unbox

()



## concat_array_trailing

In [ ]:
inl concat_array_trailing (separator : string) (input : a i32 string) =
    ("", input)
    ||> am.fold fun acc (x : string) =>
        $'!acc + !x + !separator + ""'

()



In [ ]:
// // test
// // rust=

;[
    "1"
    "2"
    "3"
]
|> fun x =>
    inl code = (a x : _ i32 _) |> concat_array_trailing "\n"
    code
    |> _assert_eq "1\n2\n3\n"

.rs output:
assert_eq / actual: "1
2
3
" / expected: "1
2
3
"




.fsx:
type Mut0 = {mutable l0 : int32; mutable l1 : string}
let rec method1 (v0 : int32, v1 : Mut0) : bool =
    let v2 : int32 = v1.l0
    let v3 : bool = v2 < v0
    v3
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "1"
    let v1 : string = "2"
    let v2 : string = "3"
    let v3 : (string []) = [|v0; v1; v2|]
    let v4 : int32 = v3.Length
    let v5 : string = ""
    let v6 : Mut0 = {l0 = 0; l1 = v5} : Mut0
    while method1(v4, v6) do
        let v8 : int32 = v6.l0
        let v9 : string = v6.l1
        let v10 : string = v3.[int v8]
        let v11 : string = "\n"
        let v12 : string = v9 + v10 + v11 + ""
        let v13 : int32 = v8 + 1
        v6.l0 <- v13
        v6.l1 <- v12
        ()
    let v14 : string = v6.l1
    let v16 : bool = v14 = "1\n2\n3\n"
    let v18 : bool =
        if v16 then
            true
        else
            method2(v16)
    let v19 : string = "assert_eq"
    let v20 : string = "1\n2\n3\n"
    let v21 : 

## concat_list_trailing

In [ ]:
inl concat_list_trailing separator input =
    ("", input)
    ||> listm.fold fun acc (x : string) =>
        $'!acc + !x + !separator + ""'

()



In [ ]:
// // test
// // rust=

[
    "1"
    "2"
    "3"
]
|> fun x =>
    inl code = (x : _) |> concat_list_trailing "\n"
    code
    |> _assert_eq "1\n2\n3\n"

.rs output:
assert_eq / actual: "1
2
3
" / expected: "1
2
3
"




.fsx:
let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = ""
    let v1 : string = "1"
    let v2 : string = "\n"
    let v3 : string = v0 + v1 + v2 + ""
    let v4 : string = "2"
    let v5 : string = v3 + v4 + v2 + ""
    let v6 : string = "3"
    let v7 : string = v5 + v6 + v2 + ""
    let v9 : bool = v7 = "1\n2\n3\n"
    let v11 : bool =
        if v9 then
            true
        else
            method1(v9)
    let v12 : string = "assert_eq"
    let v13 : string = "1\n2\n3\n"
    let v14 : string = $"{v12} / actual: %A{v7} / expected: %A{v13}"
    System.Console.WriteLine v14
    let v15 : bool = v11 = false
    if v15 then
        failwith<unit> v14
method0()


.rs:
#![allow(dead_code)]
#![allow(non_camel_case_types)]
#![allow(non_snake_case)]
#![allow(non_upper_case_globals)]
#![allow(unreachable_code)]
#![allow(unused_attributes)]
#![allow(unused_imports)]
#![allow(unused_macros)]
#![allow(unused_parens)]
#![allow(unused_variables)]
mod mo

## concat_list_heap_trailing

In [ ]:
inl concat_list_heap_trailing separator input =
    inl separator = join separator
    inl separator = separator |> as_str
    ("", input)
    ||> listm.fold fun acc (x : string) =>
        inl acc = acc |> to_std_string
        inl x = x |> as_str
        $'$"{!acc}{!x}{!separator}"'

()



In [ ]:
// // test
// // rust=

types ()

[
    "1"
    "2"
    "3"
]
|> fun x =>
    inl code = (x : _) |> concat_list_heap_trailing "\n"
    code
    |> _assert_eq "1\n2\n3\n"

.rs output:
assert_eq / actual: "1
2
3
" / expected: "1
2
3
"




.fsx:
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type ActionFn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $

## concat

In [ ]:
inl concat (a : string) (b : seq.seq' _) : string =
    inl a = join a
    b |> $"String.concat" a

()



## ellipsis

In [ ]:
inl ellipsis (max : i32) (s : string) =
    if sm.length s <= max
    then s
    else s |> slice 0 (max - 1) |> fun x => $'!x + "..."'

()



In [ ]:
// // test

"12345"
|> ellipsis 2
|> _assert_eq "12..."

"12345"
|> ellipsis 4
|> _assert_eq "1234..."

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "12"
    let v1 : string = v0 + "..."
    let v3 : bool = v1 = "12..."
    let v5 : bool =
        if v3 then
            true
        else
            method1(v3)
    let v6 : string = "assert_eq"
    let v7 : string = "12..."
    let v8 : string = $"{v6} / actual: %A{v1} / expected: %A{v7}"
    System.Console.WriteLine v8
    let v9 : bool = v5 = false
    if v9 then
        failwith<unit> v8
    let v10 : string = "1234"
    let v11 : string = v10 + "..."
    let v13 : bool = v11 = "1234..."
    let v15 : bool =
        if v13 then
            true
        else
            method1(v13)
    let v16 : string = "1234..."
    let v17 : string = $"{v6} / actual: %A{v11} / expected: %A{v16}"
    System.Console.WriteLine v17
    let v18 : bool = v15 = false
    if v18 then
        failwith<unit> v17
method0()

assert_eq / actual: "12..." / expected: "12..."
assert_eq / actual: "1234..." / expected: "1

## ellipsis_end

In [ ]:
inl ellipsis_end (max : i64) (s : string) =
    inl len = sm.length s
    if len <= max
    then s
    else
        inl half = f64 max / 2
        inl start_half = half |> math.ceil |> i64
        inl end_half = half |> math.floor |> i64
        inl start = s |> slice 0 (start_half - 1)
        inl end = s |> slice (len - end_half) (len - 1)
        (a ;[start; "..."; end] : _ i32 _)
        |> seq.of_array
        |> concat ""

()



In [ ]:
// // test

"12345"
|> ellipsis_end 2
|> _assert_eq "1...5"

"12345"
|> ellipsis_end 3
|> _assert_eq "12...5"

"1234567"
|> ellipsis_end 4
|> _assert_eq "12...67"

let rec method1 () : string =
    let v0 : string = ""
    v0
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : float = float 2L
    let v1 : float = v0 / 2.0
    let v2 : (float -> float) = ceil
    let v3 : float = v2 v1
    let v4 : int64 = int64 v3
    let v5 : (float -> float) = floor
    let v6 : float = v5 v1
    let v7 : int64 = int64 v6
    let v8 : int64 = v4 - 1L
    let v9 : string = "12345".[int 0L..int v8]
    let v10 : int64 = 5L - v7
    let v11 : string = "12345".[int v10..int 4L]
    let v12 : string = "..."
    let v13 : (string []) = [|v9; v12; v11|]
    let v14 : ((string []) -> string seq) = Seq.ofArray
    let v15 : string seq = v14 v13
    let v16 : string = method1()
    let v17 : (string -> (string seq -> string)) = String.concat
    let v18 : (string seq -> string) = v17 v16
    let v19 : string = v18 v15
    let v21 : bool = v19 = "1...5"
    let v23 : bool =
        if v21 then
            true
        else
            method2(v21

## format_ellipsis

In [ ]:
inl format_ellipsis s =
    s
    |> format_debug
    |> ellipsis_end 400

()



## split

In [ ]:
inl split (separator : string) (str : string) : array_base string =
    $"!str.Split !separator"

()



## split_string

In [ ]:
inl split_string (separator : array_base string) (str : string) : array_base string =
    run_target function
        | Fsharp (Native) => fun () => $"!str.Split (!separator, System.StringSplitOptions.None)"
        | _ => fun () => str |> split ((a separator : _ i32 _) |> seq.of_array |> concat "")

()



## join'

In [ ]:
inl join' (concat : string) (s : a i32 string) : string =
    $"System.String.Join (!concat, !s)"

()



## to_char_array

In [ ]:
inl to_char_array (str : string) : a i32 char =
    am.init (str |> sm.length) (fun i => sm.index str i)

()



In [ ]:
// // test

"abc"
|> to_char_array
|> _assert_eq (a ;['a'; 'b'; 'c'])

type Mut0 = {mutable l0 : int32}
let rec method1 (v0 : Mut0) : bool =
    let v1 : int32 = v0.l0
    let v2 : bool = v1 < 3
    v2
and method2 (v0 : (char []), v1 : (char []), v2 : int32) : bool =
    let v3 : int32 = v0.Length
    let v4 : bool = v2 < v3
    if v4 then
        let v5 : char = v0.[int v2]
        let v6 : char = v1.[int v2]
        let v7 : bool = v5 = v6
        if v7 then
            let v8 : int32 = v2 + 1
            method2(v0, v1, v8)
        else
            false
    else
        true
and method3 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (char []) = Array.zeroCreate<char> (3)
    let v1 : Mut0 = {l0 = 0} : Mut0
    while method1(v1) do
        let v3 : int32 = v1.l0
        let v4 : char = "abc".[int v3]
        v0.[int v3] <- v4
        let v5 : int32 = v3 + 1
        v1.l0 <- v5
        ()
    let v6 : (char []) = [|'a'; 'b'; 'c'|]
    let v7 : int32 = v0.Length
    let v8 : int32 = v6.Length
    let v9 : bool = v7 = v8
    let v10 : bo

## to_char_list

In [ ]:
inl to_char_list (str : string) : list char =
    listm.init (str |> sm.length) (fun (i : i64) => sm.index str i)

()



In [ ]:
// // test

"abc"
|> to_char_list
|> _assert_eq ['a'; 'b'; 'c']

type UH0 =
    | UH0_0
    | UH0_1 of char * UH0
let rec method0 () : unit =
    let v18 : string = "assert_eq"
    let v19 : UH0 = UH0_0
    let v20 : UH0 = UH0_1('c', v19)
    let v21 : UH0 = UH0_1('b', v20)
    let v22 : UH0 = UH0_1('a', v21)
    let v23 : UH0 = UH0_0
    let v24 : UH0 = UH0_1('c', v23)
    let v25 : UH0 = UH0_1('b', v24)
    let v26 : UH0 = UH0_1('a', v25)
    let v27 : string = $"{v18} / actual: %A{v22} / expected: %A{v26}"
    System.Console.WriteLine v27
    ()
method0()

assert_eq / actual: UH0_1 ('a', UH0_1 ('b', UH0_1 ('c', UH0_0))) / expected: UH0_1 ('a', UH0_1 ('b', UH0_1 ('c', UH0_0)))


In [ ]:
nominal encoding = $'System.Text.Encoding'

inl encoding_utf8 () : encoding =
    $'System.Text.Encoding.UTF8'

()



## utf8_get_bytes

In [ ]:
inl utf8_get_bytes (s : string) : a i32 u8 =
    s |> $'`encoding.UTF8.GetBytes'

()



## byte_to_string

In [ ]:
inl byte_to_string (format : string) (x : u8) : string =
    $'!x.ToString' format

()



## windows

In [ ]:
nominal windows t = $'std_slice_Windows<`t>'

inl windows (len : unativeint) (source : am'.vec u8) : windows u8 =
    inl source = source |> rust.new_box |> rust.box_leak
    // inl source' = source |> rust.clone
    inl result = !\\(len, $'"<[_]>::windows(!source, $0)"')
    // source |> rust.drop
    result

()



## any

In [ ]:
inl any forall t. (fn : string -> bool) (source : windows t) : bool =
    (!\($'"true; let mut !source = !source"') : bool) |> ignore
    inl fn' x =
        x
        |> str_from_utf8
        |> resultm.unwrap_or' #""
        |> ref_to_std_string
        |> from_std_string
        |> fn
    !\\(fn', $'"!source.any(move |x| $0(x))"')

()



## slice_contains

In [ ]:
inl slice_contains (text : string) (source : am'.vec u8) : bool =
    fun () =>
        inl source = join source
        source
        |> windows (text |> length |> (fun x => x : i32) |> unativeint)
        |> any ((=.) text)
    |> fun x => join x ()

()



## as_bytes

In [ ]:
inl as_bytes (text : string) : rust.ref' (am'.slice u8) =
    inl text = join text
    !\($'"!text.as_bytes()"')

()



## main

In [ ]:
inl main () =
    types ()
    $"let contains x = !contains x" : ()
    $"let ends_with x = !ends_with x" : ()
    $"let pad_left x = !pad_left x" : ()
    $"let pad_right x = !pad_right x" : ()
    $"let replace x = !replace x" : ()
    $"let replace_regex x = !replace_regex x" : ()
    inl slice (a : i32) (b : i32) c = slice a b c
    $"let slice x = !slice x" : ()
    $"let split x = !split x" : ()
    $"let split_string x = !split_string x" : ()
    $"let starts_with x = !starts_with x" : ()
    $"let substring x = !substring x" : ()
    $"let to_lower x = !to_lower x" : ()
    $"let to_upper x = !to_upper x" : ()
    $"let trim x = !trim x" : ()
    $"let trim_end x = !trim_end x" : ()
    $"let trim_start x = !trim_start x" : ()
    $"let ellipsis x = !ellipsis x" : ()
    $"let ellipsis_end x = !ellipsis_end x" : ()
    $"let format_exception x = !format_exception x" : ()
    $"let concat_array_trailing x = !concat_array_trailing x" : ()
    inl concat a (b : seq.seq' string) = concat a b
    $"let concat x = !concat x" : ()
    $"let join' x = !join' x" : ()
    $"let to_char_array x = !to_char_array x" : ()

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type ActionFn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $1)")>]

## to_string std_string

In [ ]:
open rust
instance to_string std_string = from_std_string

()

